- MessageID : จัดทำข้อมูล temp หรือไฟล์ที่ไม่สามารถจับคู่ได้
- MessageID : อ่านข้อมูลและจับคู่รวมถึงการ Cancel

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import uuid
from datetime import datetime
from sqlalchemy import create_engine, update, Table, Column, Integer, String, MetaData, inspect,text
import numpy as np
import json
import pyodbc

# Database configuration
db_config = {
    'dbname': 'import_final',
    'user': 'postgres',
    'password': '123',
    'host': 'localhost',
    'port': '5432'
}

postgres_db_config_mo = {
    'dbname': '67C_Monitoring',
    'user': 'postgres',
    'password': 'MyTTT%401234',
    'host': '203.154.82.165',
    'port': '5432'
}

postgres_db_config_med = {
    'dbname': 'meb',
    'user': 'postgres',
    'password': 'MyTTT%401234',
    'host': '203.154.82.165',
    'port': '5432'
}

sql_server_config_server = {
    'driver': '{SQL Server}',
    'server': '203.154.82.165',
    'database': 'response',
    'user': 'sa',
    'password': 'MyTTT@1234'
}


def setup_postgres_engine(db_config):
    try:
        db_url = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
        engine = create_engine(db_url)
        print("PostgreSQL connection successful")
        return engine
    except Exception as e:
        print(f"Error connecting to PostgreSQL: {e}")
        return None


def setup_sql_server_connection(db_config):
    try:
        conn_str = (
            f"DRIVER={db_config['driver']};"
            f"SERVER={db_config['server']};"
            f"DATABASE={db_config['database']};"
            f"UID={db_config['user']};"
            f"PWD={db_config['password']};"
        )
        conn = pyodbc.connect(conn_str)
        print("SQL Server connection successful")
        return conn
    except pyodbc.Error as e:
        print(f"Error connecting to SQL Server: {e}")
        return None
        
postgres_mybase = setup_postgres_engine(db_config)
postgres_67C_Monitoring = setup_postgres_engine(postgres_db_config_mo)
postgres_meb = setup_postgres_engine(postgres_db_config_med)
sql_server_resp = setup_sql_server_connection(sql_server_config_server)

----

In [ ]:
control_query = "SELECT * FROM import_control"
import_control_df = pd.read_sql(control_query, postgres_mybase) 

control_query = "SELECT * FROM response_import"
response_import_df = pd.read_sql(control_query, postgres_mybase) 


In [ ]:
#หา FK ของ ic_id (res_ref_id)
df_map = import_control_df.set_index('ic_reference_number')['ic_id'].to_dict()
response_import_df['res_ref_id'] = response_import_df['res_ref_no'].map(df_map)

In [ ]:
condition = df_cancel_import_not_null['cc_document_number'].isin(df_response_XCD_message_id_not_null['documentnumber'])
filtered_import_cancel_df = df_cancel_import_not_null[condition]

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['cc_is_match'] = 't' if group['cc_messageid'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['cc_is_match'] = 't' if group['cc_messageid'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'ic_reference_number'
filtered_import_cancel_df = filtered_import_cancel_df.groupby('cc_reference_number').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
mapping_send_cancel = filtered_import_cancel_df[filtered_import_cancel_df['cc_is_match'] == 't']
not_mapping_send_cancel = pd.concat([
    df_cancel_import_not_null[~condition],
    filtered_import_cancel_df[filtered_import_cancel_df['cc_is_match'] == 'f']
])

In [ ]:
# Step 1: Convert 'ic_created_date' to datetime and create 'ic_is_use' based on 'ic_ms_messageid' being not null
import_control_df['ic_created_date'] = pd.to_datetime(import_control_df['ic_created_date'], errors='coerce')
import_control_df['ic_is_use'] = import_control_df['ic_ms_messageid'].notnull()

# Step 2: Handle rows with null 'ic_ms_messageid'
empty_ic_ms_messageid = import_control_df[import_control_df['ic_ms_messageid'].isnull()]
for ic_reference_number, group in empty_ic_ms_messageid.groupby('ic_reference_number'):
    if len(group) == 1:
        import_control_df.loc[group.index, 'ic_is_use'] = True
    else:
        most_recent_index = group['ic_created_date'].idxmax()
        import_control_df.loc[group.index, 'ic_is_use'] = False
        import_control_df.loc[most_recent_index, 'ic_is_use'] = True

# Step 3: Convert boolean 'ic_is_use' to string 't' or 'f'
import_control_df['ic_is_use'] = import_control_df['ic_is_use'].replace({True: 't', False: 'f'})

# Step 4: Handle duplicates and mismatches
duplicates = import_control_df[import_control_df.duplicated('ic_reference_number', keep=False)]
for ic_reference_number, group in duplicates.groupby('ic_reference_number'):
    if group['ic_ms_messageid'].isnull().any() and group['ic_ms_messageid'].notnull().any():
        import_control_df.loc[group[group['ic_ms_messageid'].isnull()].index, 'ic_is_use'] = 'f'


In [ ]:
# Step 5: Match 'ic_ms_messageid' with 'res_message_id' in response_import_df
xml_message_ids = set(response_import_df[response_import_df['res_message_id'].notnull()]['res_message_id'])
import_control_df.loc[import_control_df['ic_ms_messageid'].isin(xml_message_ids), 'ic_is_use'] = 't'

# Handle cases with duplicate 'ic_reference_number' and 'res_message_type' being 'IDCA'
duplicate_refs = import_control_df[import_control_df.duplicated('ic_reference_number', keep=False)]['ic_reference_number'].unique()
filtered_response_import = response_import_df[response_import_df['res_message_type'] == 'IDCA']

# Check for matches and update 'ic_is_use'
def check_match(row):
    if row['ic_reference_number'] in duplicate_refs and row['ic_ms_messageid']:
        match = filtered_response_import[
            (filtered_response_import['res_ref_no'] == row['ic_reference_number']) &
            (filtered_response_import['res_message_id'] == row['ic_ms_messageid'])
        ]
        return 't' if not match.empty else 'f'
    return row['ic_is_use']

import_control_df['ic_is_use'] = import_control_df.apply(check_match, axis=1)


---

source XML (send)

In [ ]:
control_query = "SELECT * FROM import_control"

med_query = """SELECT "fileName", "messageId" FROM meb_sent"""

import_control_df = pd.read_sql(control_query, postgres_mybase) 
med_df = pd.read_sql(med_query, postgres_meb)

In [ ]:
# Create the mapping dictionary
ms_filename_to_id = med_df.set_index('fileName')['messageId'].to_dict()
import_control_df['ic_ms_messageid'] = import_control_df['ic_filename'].str.replace(r'\.sgn$', '', regex=True).map(ms_filename_to_id)
import_control_df['ic_filename'] = import_control_df['ic_filename'].str.replace(r'\.sgn$', '', regex=True) + '.sgn'

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['is_match'] = 't' if group['ic_ms_messageid'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['is_match'] = 't' if group['ic_ms_messageid'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'ic_reference_number'
import_control_df = import_control_df.groupby('ic_reference_number').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
import_control_df_not_null = import_control_df[import_control_df['is_match'] == 't']
import_control_df_null = import_control_df[import_control_df['is_match'] == 'f']


In [ ]:
# กรองหา 'ic_reference_number' ที่ซ้ำกันและไม่มี 'ic_ms_messageid' สักตัว (ทุกค่าเป็น null)
grouped = import_control_df_not_null.groupby('ic_reference_number')

duplicatenot_import_control_df_not_null = grouped.filter(
    lambda x: len(x) > 1 and x['ic_ms_messageid'].isnull().all()
)['ic_reference_number'].unique()

print("Duplicate 'ic_reference_number' without any 'ic_ms_messageid':")
print(duplicatenot_import_control_df_not_null)


-----

source XML (Resonse)

In [ ]:
query_sql_resp = """
        SELECT "FileName", "InboxId" 
        FROM response;"""

query_meb_inbox = """
        SELECT "id", "reftomessageid", "messagetimestamp" 
        FROM meb_inbox;"""

res_query = """SELECT *,  res_data->>'DocumentNumber' AS DocumentNumber FROM response_import;""" 

df_sql_resp = pd.read_sql(query_sql_resp, sql_server_resp)
df_response_import = pd.read_sql(res_query, postgres_mybase) 
df_meb_inbox = pd.read_sql(query_meb_inbox, postgres_meb)

df_sql_resp = df_sql_resp.astype(str)
df_meb_inbox = df_meb_inbox.astype(str)

In [ ]:
# Create the mapping dictionary
map_inbox_id = df_sql_resp.set_index('FileName')['InboxId'].to_dict()
df_response_import['res_inbox_id'] = df_response_import['res_filename'].map(map_inbox_id)

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['is_match'] = 't' if group['res_inbox_id'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['is_match'] = 't' if group['res_inbox_id'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'res_ref_no'
df_response_import = df_response_import.groupby('res_ref_no').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
df_response_import_inbox_id_not_null = df_response_import[df_response_import['is_match'] == 't']
df_response_import_inbox_id_null = df_response_import[df_response_import['is_match'] == 'f']

In [ ]:
# Create the mapping dictionary
map_message_id = df_meb_inbox.set_index('id')['reftomessageid'].to_dict()
df_response_import_inbox_id_not_null['res_message_id'] = df_response_import_inbox_id_not_null['res_inbox_id'].map(map_message_id)

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['is_match'] = 't' if group['res_message_id'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['is_match'] = 't' if group['res_message_id'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'res_ref_no'
df_response_import_inbox_id_not_null = df_response_import_inbox_id_not_null.groupby('res_ref_no').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
df_response_import_message_id_not_null = df_response_import_inbox_id_not_null[df_response_import_inbox_id_not_null['is_match'] == 't']
df_response_import_message_id_null = df_response_import_inbox_id_not_null[df_response_import_inbox_id_not_null['is_match'] == 'f']

-------

Mapping messageid

In [ ]:
#Filter import_control_df_not_null based on the condition
condition = (
    import_control_df_not_null['ic_ms_messageid'].isin(df_response_import_message_id_not_null['res_message_id']) &
    import_control_df_not_null['ic_reference_number'].isin(df_response_import_message_id_not_null['res_ref_no'])
)

# Applying the condition to filter rows
filtered_import_control_df = import_control_df_not_null[condition]

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['ic_is_match'] = 't' if group['ic_ms_messageid'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['ic_is_match'] = 't' if group['ic_ms_messageid'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'ic_reference_number'
filtered_import_control_df = filtered_import_control_df.groupby('ic_reference_number').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
mapping_send = filtered_import_control_df[filtered_import_control_df['ic_is_match'] == 't']
not_mapping_send = pd.concat([
    import_control_df_not_null[~condition],
    filtered_import_control_df[filtered_import_control_df['ic_is_match'] == 'f']
])



In [ ]:
not_mapping_send_import = not_mapping_send.copy()

In [ ]:
# กรองหา 'ic_reference_number' ที่ซ้ำกันและไม่มี 'ic_ms_messageid' สักตัว (ทุกค่าเป็น null)
grouped = mapping_send.groupby('ic_reference_number')

duplicatenot_mapping_send = grouped.filter(
    lambda x: len(x) > 1 and x['ic_ms_messageid'].isnull().all()
)['ic_reference_number'].unique()

print("Duplicate 'ic_reference_number' without any 'ic_ms_messageid':")
print(duplicatenot_mapping_send)


In [ ]:
# Filter import_control_df_not_null based on the condition
condition = (
    df_response_import_message_id_not_null['res_message_id'].isin(import_control_df_not_null['ic_ms_messageid']) &
    df_response_import_message_id_not_null['res_ref_no'].isin(import_control_df_not_null['ic_reference_number'])
)

# Applying the condition to filter rows
filtered_response_import_df = df_response_import_message_id_not_null[condition]

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['res_is_match'] = 't' if group['res_message_id'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['res_is_match'] = 't' if group['res_message_id'].notnull().any() else 'f'
    return group

filtered_response_import_df = filtered_response_import_df.groupby('res_ref_no').apply(check_group).reset_index(drop=True)

mapping_response = filtered_response_import_df[filtered_response_import_df['res_is_match'] == 't']
not_mapping_response = pd.concat([
    df_response_import_message_id_not_null[~condition],
    filtered_response_import_df[filtered_response_import_df['res_is_match'] == 'f']
])

In [ ]:
not_mapping_response_import = not_mapping_response.copy()

In [ ]:
# กรองหา 'res_ref_no' ที่ซ้ำกันและไม่มี 'res_message_id' สักตัว (ทุกค่าเป็น null)
grouped = not_mapping_response.groupby('res_ref_no')

duplicatenot_mapping_response = grouped.filter(
    lambda x: len(x) > 1 and x['res_message_id'].isnull().all()
)['res_ref_no'].unique()

print("Duplicate 'res_ref_no' without any 'res_message_id':")
print(duplicatenot_mapping_response)


---
----

Declaration No. (PMTA,PMTR) : หา ic_rs_id กับ ic_declaration_no ไป update import_control(send)

In [ ]:
query = """
        SELECT rs_id, rs_type, rs_message, rs_status_id, rs_status_priority 
        FROM response_status
        WHERE rs_declaration_type = 'Import';"""
        
df_response_status = pd.read_sql(query, postgres_67C_Monitoring)


In [ ]:
df_response_PAT = df_response_import[df_response_import['res_message_type'].isin(['PMTA', 'PMTR'])]

# rs_type, rs_message จับคุ่และมีค่าซ้ำกันไปดูที่ rs_status_priority เอามากที่สุด
df_response_status = df_response_status[df_response_status['rs_type'].isin(['PMTA', 'PMTR'])] 
df_response_status = df_response_status.loc[df_response_status.groupby(['rs_type', 'rs_message'])['rs_status_priority'].idxmax()]

merged_status = pd.merge(df_response_PAT, df_response_status, 
                        left_on=['res_message_type', 'res_message'], 
                        right_on=['rs_type', 'rs_message'],
                        how='left') 

# ใช้ groupby ตาม res_ref_no และเอาเพื่อเลือกค่าแรกที่เจอ ถ้า rs_status_priority เท่ากัน 
ic_rs_id_map = merged_status.groupby('res_ref_no')['rs_id'].first()
ic_declaration_no_map = merged_status.groupby('res_ref_no')['res_declaration_number'].first()

import_control_df['ic_rs_id'] = import_control_df['ic_reference_number'].map(ic_rs_id_map)
import_control_df['ic_declaration_no'] = import_control_df['ic_reference_number'].map(ic_declaration_no_map)


----------

cancel : ส่ง หา filename จาก cancel ไปหาที่ meb_sent เพื่อเอา messagid


In [ ]:
res_query = """ SELECT * 
                FROM cancel_09 
                WHERE cc_declaration_type = 'IMP';"""
df_cancel_import = pd.read_sql(res_query, postgres_mybase) 

In [ ]:
# Create the mapping dictionary
ms_filename_to_id = med_df.set_index('fileName')['messageId'].to_dict()
df_cancel_import['cc_messageid'] = df_cancel_import['cc_filename'].str.replace(r'\.sgn$', '', regex=True).map(ms_filename_to_id)
df_cancel_import['cc_filename'] = df_cancel_import['cc_filename'].str.replace(r'\.sgn$', '', regex=True) + '.sgn'

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['is_match'] = 't' if group['cc_messageid'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['is_match'] = 't' if group['cc_messageid'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'cc_reference_number'
df_cancel_import = df_cancel_import.groupby('cc_reference_number').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
df_cancel_import_not_null = df_cancel_import[df_cancel_import['is_match'] == 't']
df_cancel_import_null = df_cancel_import[df_cancel_import['is_match'] == 'f']


responce (XDCA,XCDR) : ส่ง filename ไปหาที่ response(sql server) ได้ InboxId --> ส่ง InboxId ไปหาที่ meb_inbox ได้ res_message_id

In [ ]:
df_response_XCD = df_response_import[df_response_import['res_message_type'].isin(['XCDA', 'XCDR'])]
map_inbox_id = df_sql_resp.set_index('FileName')['InboxId'].to_dict()
df_response_XCD['res_inbox_id'] = df_response_XCD['res_filename'].map(map_inbox_id)

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['is_match'] = 't' if group['res_inbox_id'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['is_match'] = 't' if group['res_inbox_id'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'res_ref_no'
df_response_XCD = df_response_XCD.groupby('res_ref_no').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
df_response_XCD_inbox_id_not_null = df_response_XCD[df_response_XCD['is_match'] == 't']
df_response_XCD_inbox_id_null = df_response_XCD[df_response_XCD['is_match'] == 'f']


In [ ]:
# Create the mapping dictionary
map_message_id = df_meb_inbox.set_index('id')['reftomessageid'].to_dict()
df_response_XCD_inbox_id_not_null['res_message_id'] = df_response_XCD_inbox_id_not_null['res_inbox_id'].map(map_message_id)

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['is_match'] = 't' if group['res_message_id'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['is_match'] = 't' if group['res_message_id'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'res_ref_no'
df_response_XCD_inbox_id_not_null = df_response_XCD_inbox_id_not_null.groupby('res_ref_no').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
df_response_XCD_message_id_not_null = df_response_XCD_inbox_id_not_null[df_response_XCD_inbox_id_not_null['is_match'] == 't']
df_response_XCD_message_id_null = df_response_XCD_inbox_id_not_null[df_response_XCD_inbox_id_not_null['is_match'] == 'f']

In [ ]:
grouped = df_response_XCD_message_id_not_null.groupby('res_ref_no')

duplicatenot_XCD_message_id = grouped.filter(
    lambda x: len(x) > 1 and x['res_message_id'].isnull().all()
)['res_ref_no'].unique()

print("Duplicate 'res_ref_no' without any 'res_message_id':")
print(duplicatenot_XCD_message_id)


----------

map send_cancel and response_cancel

In [ ]:
condition = df_cancel_import_not_null['cc_document_number'].isin(df_response_XCD_message_id_not_null['documentnumber'])
filtered_import_cancel_df = df_cancel_import_not_null[condition]

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['cc_is_match'] = 't' if group['cc_messageid'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['cc_is_match'] = 't' if group['cc_messageid'].notnull().any() else 'f'
    return group

# Apply the function to each group by 'ic_reference_number'
filtered_import_cancel_df = filtered_import_cancel_df.groupby('cc_reference_number').apply(check_group).reset_index(drop=True)

# Initialize lists to hold grouped dataframes
mapping_send_cancel = filtered_import_cancel_df[filtered_import_cancel_df['cc_is_match'] == 't']
not_mapping_send_cancel = pd.concat([
    df_cancel_import_not_null[~condition],
    filtered_import_cancel_df[filtered_import_cancel_df['cc_is_match'] == 'f']
])

In [ ]:
grouped = mapping_send_cancel.groupby('cc_reference_number')

duplicatenot_mapping_response = grouped.filter(
    lambda x: len(x) > 1 and x['cc_messageid'].isnull().all()
)['cc_reference_number'].unique()

print("Duplicate 'res_ref_no' without any 'cc_messageid':")
print(duplicatenot_mapping_response)


In [ ]:
condition = df_response_XCD_message_id_not_null['documentnumber'].isin(df_cancel_import_not_null['cc_document_number'])
filtered_cancel_response = df_response_XCD_message_id_not_null[condition]

# Function to check each group
def check_group(group):
    if len(group) == 1:  # Unique reference number
        group['res_is_match'] = 't' if group['res_message_id'].notnull().all() else 'f'
    else:  # Duplicate reference number
        group['res_is_match'] = 't' if group['res_message_id'].notnull().any() else 'f'
    return group

filtered_cancel_response = filtered_cancel_response.groupby('res_ref_no').apply(check_group).reset_index(drop=True)

mapping_response_cancel = filtered_cancel_response[filtered_cancel_response['res_is_match'] == 't']
not_mapping_response_cancel = pd.concat([
    df_response_XCD_message_id_not_null[~condition],
    filtered_cancel_response[filtered_cancel_response['res_is_match'] == 'f']
])

In [ ]:
grouped = mapping_response_cancel.groupby('res_ref_no')

duplicatenot_mapping_response = grouped.filter(
    lambda x: len(x) > 1 and x['res_message_id'].isnull().all()
)['res_ref_no'].unique()

print("Duplicate 'res_ref_no' without any 'res_message_id':")
print(duplicatenot_mapping_response)


-------

update ic_is_cancel กับ ic_cancel_reason

In [ ]:
import_control_df.loc[import_control_df['ic_declaration_no'].isin(df_response_XCD_message_id_not_null['documentnumber']), 'ic_is_cancel'] = 't'

# mapping จาก res_message_id ไปยัง cc_cancel_reason
res_to_reason = df_cancel_import.set_index('cc_messageid')['cc_cancel_reason'].to_dict()

# mapping จาก documentnumber ไปยัง res_message_id
doc_to_res = df_response_XCD_message_id_not_null.set_index('documentnumber')['res_message_id'].to_dict()

# Update ic_cancel_reason สำหรับแถวที่ ic_is_cancel เป็น 't'
import_control_df.loc[import_control_df['ic_is_cancel'] == 't', 'ic_cancel_reason'] = import_control_df['ic_declaration_no'].map(doc_to_res).map(res_to_reason)

---

Declaration No. (IDCA,IDCA) : หา ic_rs_id กับ ic_declaration_no ไป update import_control(send)

In [ ]:
query = """
        SELECT rs_id, rs_type, rs_message, rs_status_id, rs_status_priority 
        FROM response_status
        WHERE rs_declaration_type = 'Import';"""
        
df_response_status = pd.read_sql(query, postgres_67C_Monitoring)

In [ ]:
df_response_IDA = df_response_import[df_response_import['res_message_type'].isin(['IDCA', 'IDCR'])]

# rs_type, rs_message จับคุ่และมีค่าซ้ำกันไปดูที่ rs_status_priority เอามากที่สุด
df_response_status = df_response_status[df_response_status['rs_type'].isin(['IDCA', 'IDCR'])] 
df_response_status = df_response_status.loc[df_response_status.groupby(['rs_type', 'rs_message'])['rs_status_priority'].idxmax()]

merged_status = pd.merge(df_response_IDA, df_response_status, 
                        left_on=['res_message_type', 'res_message'], 
                        right_on=['rs_type', 'rs_message'],
                        how='left') 

# ใช้ groupby ตาม res_ref_no และเอาเพื่อเลือกค่าแรกที่เจอ ถ้า rs_status_priority เท่ากัน 
ic_rs_id_map = merged_status.groupby('res_ref_no')['rs_id'].first()
ic_declaration_no_map = merged_status.groupby('res_ref_no')['res_declaration_number'].first()

import_control_df['ic_rs_id'] = import_control_df['ic_reference_number'].map(ic_rs_id_map)
import_control_df['ic_declaration_no'] = import_control_df['ic_reference_number'].map(ic_declaration_no_map)


-----

In [ ]:
# import_control_df_null 13963

# df_response_import_inbox_id_null 0
# df_response_import_message_id_null 61654

# not_mapping_send 15
# not_mapping_response 1860

In [356]:
required_col = ['eh_document_name', 'eh_filename', 'eh_type', 'eh_message_id', 'eh_file_date', 
                'eh_folder', 'eh_message', 'eh_is_use', 'eh_is_match', 'eh_ref_no', 'eh_remark']
                
control = pd.DataFrame(columns=required_col)
inbox = pd.DataFrame(columns=required_col)
message_id = pd.DataFrame(columns=required_col)
not_mapping_send = pd.DataFrame(columns=required_col)
not_mapping_response = pd.DataFrame(columns=required_col)

def update_dataframe(df, values_dict):
    new_df = pd.DataFrame(columns=df.columns)
    for col, value in values_dict.items():
        if value is not None:
            new_df[col] = value
    return new_df

In [357]:
control_values = {
    'eh_document_name' : import_control_df_null['ic_doc_name'],
    'eh_filename': import_control_df_null['ic_filename'],
    'eh_message_id': import_control_df_null['ic_ms_messageid'],
    'eh_file_date': import_control_df_null['ic_created_date'],
    'eh_ref_no': import_control_df_null['ic_reference_number'],
    'eh_type': 'SEND',
    'eh_is_use': 'f',
    'eh_remark': 'ไม่พบ MessageId ที่ Meb_Send'
}

inbox_values = {
    'eh_document_name' : df_response_import_inbox_id_null['res_message_type'],
    'eh_filename': df_response_import_inbox_id_null['res_filename'],
    'eh_message_id': df_response_import_inbox_id_null['res_message_id'],
    'eh_file_date': df_response_import_inbox_id_null['res_datetime'],
    'eh_ref_no': df_response_import_inbox_id_null['res_ref_no'],
    'eh_type': 'RESPONSE',
    'eh_is_use': 'f',
    'eh_remark': 'ไม่พบ InboxId ที่ Reponse'
}

message_id_values = {
    'eh_document_name' : df_response_import_message_id_null['res_message_type'],
    'eh_filename': df_response_import_message_id_null['res_filename'],
    'eh_message_id': df_response_import_message_id_null['res_message_id'],
    'eh_file_date': df_response_import_message_id_null['res_datetime'],
    'eh_ref_no': df_response_import_message_id_null['res_ref_no'],
    'eh_type': 'RESPONSE',
    'eh_is_use': 'f',
    'eh_remark': 'ไม่พบ MessageId ที่ Meb_Inbox'
}

not_mapping_send_values = {
    'eh_document_name' : not_mapping_send_import['ic_doc_name'],
    'eh_filename': not_mapping_send_import['ic_filename'],
    'eh_message_id': not_mapping_send_import['ic_ms_messageid'],
    'eh_file_date': not_mapping_send_import['ic_created_date'],
    'eh_ref_no': not_mapping_send_import['ic_reference_number'],
    'eh_type': 'SEND',
    'eh_is_use': 'f',
    'eh_remark': 'ไม่พบ MessageId ที่ตรงกัน'
}

not_mapping_response_values = {
    'eh_document_name' : not_mapping_response_import['res_message_type'],
    'eh_filename': not_mapping_response_import['res_filename'],
    'eh_message_id': not_mapping_response_import['res_message_id'],
    'eh_file_date': not_mapping_response_import['res_datetime'],
    'eh_ref_no': not_mapping_response_import['res_ref_no'],
    'eh_type': 'RESPONSE',
    'eh_is_use': 'f',
    'eh_remark': 'ไม่พบ RefMessageId ที่ตรงกัน '
}

control = update_dataframe(control, control_values)
inbox = update_dataframe(inbox, inbox_values)
message_id = update_dataframe(message_id, message_id_values)
not_mapping_send = update_dataframe(not_mapping_send, not_mapping_send_values)
not_mapping_response = update_dataframe(not_mapping_response, not_mapping_response_values)

In [362]:
# control.to_sql('exception_handle', postgres_mybase, if_exists='append', index=False)
# inbox.to_sql('exception_handle', postgres_mybase, if_exists='append', index=False)
# message_id.to_sql('exception_handle', postgres_mybase, if_exists='append', index=False)
# not_mapping_send.to_sql('exception_handle', postgres_mybase, if_exists='append', index=False)
# not_mapping_response.to_sql('exception_handle', postgres_mybase, if_exists='append', index=False)

162